In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [39]:
%load_ext autoreload
%autoreload
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

from datasets.kitti_raw_monosf import KITTI_Raw_KittiSplit_Train, KITTI_Raw_KittiSplit_Valid
from models.CNet import CNet
from augmentations import Augmentation_SceneFlow

from losses import _adaptive_disocc_detection

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
%autoreload
args = {
    'batch_size' : 1,
    'num_workers' : 2,
    'epochs': 1, 
    'lr': 2e-4,
    'momentum': 0.9,
    'beta': 0.999,
    'weight_decay': 0.0,
    'train': True,
    'cuda': True,
    'debug': False,
    'evaluation': False,
    'finetuning': False,
}

data_root = '/external/datasets/kitti_data_jpg/'
train_dataset = KITTI_Raw_KittiSplit_Train(args={}, root=data_root)
train_loader = DataLoader(train_dataset, batch_size=args['batch_size'], num_workers=args['num_workers'], shuffle=True, pin_memory=True)
# val_dataset = KITTI_Raw_KittiSplit_Valid(args={}, root=data_root)
# val_loader = DataLoader(val_dataset, batch_size=args['batch_size'], num_workers=args['num_workers'], shuffle=False, pin_memory=True)

In [3]:
%autoreload 

augmentation = Augmentation_SceneFlow(args)
model = CNet(args)

if args['cuda']:
    augmentation = augmentation.cuda()
    model = model.cuda()
    
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_params)

optimizer = Adam(model.parameters(), lr=args['lr'], betas=[args['momentum'], args['beta']], weight_decay=args['weight_decay'])

72686514


In [4]:
# DEBUG
import pynvml
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total / (1024**3)} Gb')
print(f'free     : {info.free / (1024**3)} Gb')
print(f'used     : {info.used / (1024**3)} Gb')

total    : 7.92620849609375 Gb
free     : 6.1063232421875 Gb
used     : 1.81988525390625 Gb


In [5]:
%autoreload
start = time.time()
for i, data_dict in enumerate(train_loader):
    # Get input and target tensor keys
    input_keys = list(filter(lambda x: "input" in x, data_dict.keys()))
    target_keys = list(filter(lambda x: "target" in x, data_dict.keys()))
    tensor_keys = input_keys + target_keys

    # Possibly transfer to Cuda
    if args['cuda']:
        for key, value in data_dict.items():
            if key in tensor_keys:
                data_dict[key] = value.cuda(non_blocking=True)

    if augmentation is not None:
        with torch.no_grad():
            data_dict = augmentation(data_dict)
        
    x = model(data_dict)
    if i == 9:
        print(f"5 iterations took {time.time() - start}s")
        break

5 iterations took 4.380586862564087s


In [33]:
flow_s = torch.rand((1, 2, 3, 3))
flow_d = torch.rand((1, 2, 3, 3))
flow_cam = torch.rand((1, 2, 3, 3))
rigidity_mask = torch.randint(0, 2, (1, 1, 3, 3)).type_as(flow_s)

loss_static = torch.norm(flow_s * rigidity_mask, p=2)
loss_dynamic = torch.norm(flow_d * (1-rigidity_mask), p=2)
loss_static + loss_dynamic

torch.norm(flow_cam-flow_s, dim=1)

tensor([[[[0.0888, 0.8093, 0.5732],
          [0.3487, 0.5563, 0.5801],
          [0.8364, 0.3219, 0.6261]]]])
tensor([[[0.0888, 0.8093, 0.5732],
         [0.3487, 0.5563, 0.5801],
         [0.8364, 0.3219, 0.6261]]])


In [78]:
from utils.sceneflow_util import projectSceneFlow2Flow

# mask loss function

# optical flow losses
optical_flows = [projectSceneFlow2Flow(data_dict['input_k_l1'], sf, disp) for (sf, disp) in zip(x['flow_f'], x['disp_l1'])]

[torch.Size([1, 2, 256, 832]), torch.Size([1, 2, 128, 416]), torch.Size([1, 2, 64, 208]), torch.Size([1, 2, 32, 104]), torch.Size([1, 2, 16, 52])]


In [45]:
# create consistency loss functions
if data_dict:
    del data_dict
if x:
    del x
torch.cuda.empty_cache()

In [55]:
def train_one_epoch(model, dataloader, optimizer, augmentations):

  model.train()

  for i, data in enumerate(dataloader):
    continue

  return

In [58]:
for epoch in range(args['epochs']):
    loss_one_epoch = train_one_epoch(model, dataloader, optimizer, augmentations=augmentation)
    optimizer.zero_grad()
    loss_one_epoch.backward()
    

NameError: name 'optimizer' is not defined